In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/04 22:03:00 WARN Utils: Your hostname, ROG-STRIX resolves to a loopback address: 127.0.1.1; using 172.27.115.120 instead (on interface eth0)
23/03/04 22:03:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/04 22:03:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
df_green.registerTempTable('green')

In [5]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [6]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-04 18:00:00| 244|308.65000000000003|            13|
|2020-01-01 00:00:00| 255| 666.3399999999999|            28|
|2020-01-31 23:00:00|  61|            128.55|             6|
|2020-01-24 13:00:00| 130|            338.27|            13|
|2020-01-16 22:00:00|  41| 433.8700000000001|            33|
|2020-01-30 07:00:00| 197|             88.41|             2|
|2020-01-04 21:00:00|  17|             93.77|             6|
|2020-01-16 19:00:00|  82| 778.5699999999997|            57|
|2020-01-19 18:00:00|  41| 578.6900000000003|            46|
|2020-01-22 14:00:00| 177|              70.4|             2|
|2020-01-08 18:00:00|  17|126.82999999999998|             5|
|2020-01-27 07:00:00| 244|241.51000000000005|            14|
|2020-01-03 06:00:00|  74| 327.8600000000001|            28|
|2020-01-21 07:00:00|  5

In [7]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

23/03/04 22:03:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/03/04 22:03:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/03/04 22:03:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/03/04 22:03:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/04 22:03:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/03/04 22:03:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/04 22:03:20 WARN MemoryManager: Total allocation exceeds 95.

In [8]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [9]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [10]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

23/03/04 22:03:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/03/04 22:03:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/03/04 22:03:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/03/04 22:03:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/04 22:03:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/03/04 22:03:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/04 22:03:28 WARN MemoryManager: Total allocation exceeds 95.

In [11]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [12]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [13]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [14]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

23/03/04 22:05:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/03/04 22:05:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/03/04 22:05:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/03/04 22:05:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/04 22:05:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/03/04 22:05:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/04 22:05:25 WARN MemoryManager: Total allocation exceeds 95.

In [15]:
df_join = spark.read.parquet('data/report/revenue/total')

In [16]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [21]:
df_join.show()

+-------------------+----+-----------------+--------------------+------------------+---------------------+
|               hour|zone|     green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+-----------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  32|68.94999999999999|                   2|              18.0|                    1|
|2020-01-01 00:00:00|  79|             null|                null| 12573.81000000003|                  721|
|2020-01-01 00:00:00|  83|             94.1|                   7|               9.8|                    1|
|2020-01-01 00:00:00| 130|           133.35|                   7|              null|                 null|
|2020-01-01 00:00:00| 142|             null|                null| 9252.300000000021|                  488|
|2020-01-01 00:00:00| 159|             13.8|                   1|              58.8|                    2|
|2020-01-01 00:00:00| 210|           

In [17]:
df_zones = spark.read.parquet('zones/')

In [20]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [18]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [22]:
df_result.show()

+-------------------+----+-----------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|     green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+-----------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|  32|68.94999999999999|                   2|              18.0|                    1|        32|    Bronx|           Bronxdale|   Boro Zone|
|2020-01-01 00:00:00|  79|             null|                null| 12573.81000000003|                  721|        79|Manhattan|        East Village| Yellow Zone|
|2020-01-01 00:00:00|  83|             94.1|                   7|               9.8|                    1|        83|   Queens|    Elmhurst/Maspeth|   Boro Zone|
|2020-01-01 00:00:00| 130|  

In [19]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

23/03/04 22:05:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/03/04 22:05:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/03/04 22:05:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/03/04 22:05:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/04 22:05:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers


23/03/04 22:05:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/04 22:05:59 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/03/04 22:05:59 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/03/04 22:05:59 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/03/04 22:05:59 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [23]:
df_result.show()

+-------------------+----+-----------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|     green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+-----------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|  32|68.94999999999999|                   2|              18.0|                    1|        32|    Bronx|           Bronxdale|   Boro Zone|
|2020-01-01 00:00:00|  79|             null|                null| 12573.81000000003|                  721|        79|Manhattan|        East Village| Yellow Zone|
|2020-01-01 00:00:00|  83|             94.1|                   7|               9.8|                    1|        83|   Queens|    Elmhurst/Maspeth|   Boro Zone|
|2020-01-01 00:00:00| 130|  